In [100]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd

import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from pymongo import MongoClient

In [101]:
# I need to update the documents, to add a field called location, that includes the point type and coordinates for each document.


# Connect to the MongoDB server
client = MongoClient("localhost:27017")
db = client['geospatial']

# Select the database and collection


# Update all documents to add the geometry field
dict_of_places = {'bar':13003,
                   'physiotherapy':15026,
                   'social_club':12097,
                   'tennis_store':17130,
                   'residential':12094,
                   'park': 16032,
                   'university': 12013,
                   'gym':18021,
                   'parking':19020,
                   'public_transport':19054,
                   'tennis_court':18047
                    }
# I loop through all the names of my collections in mongo.
for key in dict_of_places.keys():
    collection = db[key]
    for document in collection.find():
        lat = document["geocodes"]["main"]['latitude']
        lng = document["geocodes"]["main"]['longitude']
        geometry = {
            "type": "Point",
            "coordinates": [lng, lat]
        }
        collection.update_one({"_id": document["_id"]}, {"$set": {"geometry": geometry}})

In [102]:
client = MongoClient("localhost:27017")
db = client['geospatial']

In [103]:
districts = db.get_collection('districts_full_equip')

In [104]:
# I am going to loop through all the documents of each collection, and see what is the district containing the highest number of places.
def what_district (location):
    what_district = districts.find_one(
                                {"geometry":
                                    {"$geoIntersects": 
                                        {"$geometry": 
                                            location}}},
                            projection = {'abb_name':1})['abb_name']
    return what_district

In [113]:
dict_districts={}
dict_category={}
for key in dict_of_places.keys():
    collection=db[key]
    dict_category[key]=0
  
    for document in collection.find():
        geometry=document['geometry']
        district_found = what_district(geometry)
        print(district_found)
        dict_category[key]+=1
        dict_districts[district_found]=dict_category[key]
    

Sydney
Inner West
Sydney
Sydney
Sydney
Sydney
Sydney
Sydney
Sydney
Inner West
Woollahra
Randwick
Sydney
Inner West
Sydney
Inner West
Waverley
Burwood
Sydney
Sydney
Sydney
Sydney
Sydney
Inner West
Sydney
Willoughby
Unincorporated - Sydney Harbour Area
Sydney
Bayside
Sydney
Ryde
Fairfield
Hornsby
Canada Bay
Burwood
Canterbury-Bankstown
Inner West
Inner West
Inner West
Sydney
North Sydney
North Sydney
Sydney
Sydney
Sydney
Bayside
Sydney
Sydney
Waverley
Sutherland
Penrith
Parramatta
Cumberland
Parramatta
Parramatta
Ryde
The Hills
Canterbury-Bankstown
Canterbury-Bankstown
Hornsby
Unincorporated - Sydney Harbour Area
Canada Bay
Inner West
Northern Beaches
The Hills
Northern Beaches
Parramatta
Canada Bay
Waverley
Ryde
Fairfield
Parramatta
Cumberland
Sutherland
Blacktown
Penrith
Ryde
Northern Beaches
Canterbury-Bankstown
Burwood
Fairfield
Canterbury-Bankstown
Willoughby
Inner West
Georges River
Sydney
Georges River
Sydney
Georges River
Sydney
Ryde
Cumberland
Inner West
Willoughby
Lane Cove
Par

In [ ]:
collection_district=db.get_collection('bar')
collection_district.find_one()['geometry']

{'type': 'Point', 'coordinates': [-33.857721, 151.214458]}

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'geospatial'), 'bar.geometry')
